# Sentinel-1 ship detection using Google Earth Engine

This tutorial uses Sentinel-2 SAR imagery to detect ships in the port of Panama City waiting to use the Panama Canal.

It uses the Google Earth Engine (GEE) API package for python. This allows to compute large amount of satellite (and other remote sensing) data without having to download the huge data sets. To be able to use this package a GEE account is required.

### Start a GEE session

The first time the "ee" package is used, you need to run ee.Authenticate(). This will open a window on the web where you have to log in with your GEE credentials. This will create an access token that needs to be pasted in the box that appeared below (this needs to be done at the beginning or when the kernel/session is restarted). To start the connection with your GEE account, you run ee.Initialize(). If you have only one project on your GEE account, you do not need to specify more. If you have multiple projects, you can specify with ee.Initialize(project=project-number). To find the project number, you have to log in to GEE online and click on the respective project.

In [2]:
# Load the packages
import ee       # GEE API package
import geemap   # package for interactive plotting->does not work on PyCharm

# Login with the GEE credentials and connect to your account
ee.Authenticate()   # needs to be done once in a while
ee.Initialize()     # starts the connection to your GEE account and allows you to use all the datasets you might have stored there

### Select the area of interest

We select a polygon at the port of Panama City. As we only want to detect the ships and not built up areas we should make sure to only select area in the ocean.

In [3]:
# Area of interest polygon at the port in Panama City
AOI = ee.Geometry.Polygon([[-79.5318588873829,8.929408571356298],[-79.57855078191415,8.87378180348808],
                           [-79.67330786199227,8.839180410557434],[-79.69734045476571,8.784218582482216],
                           [-79.56893774480477,8.701421206774654],[-79.37155075269101,8.7502874576597],
                           [-79.36262436108944,8.833073944690653],[-79.39077682690976,8.913128442052326],
                           [-79.49034042554257,8.957897048024819],[-79.52055282788632,8.943653088344814],
                           [-79.5318588873829,8.929408571356298]])

Let's visualize the general area of interest at the entry to the Panama Canal (as a true color optical satellite image) and the polygone we just defined in yellow. You will notice that there are some small islands within the selected AOI (this will have an influence later on).

In [4]:
# Initialize the map
Map = geemap.Map()

# Define how the basemap should be displayed
Map.set_center(-79.529382, 8.897607, 11) # coordinates and zoom
Map.add_basemap('SATELLITE')             # background map as optical true color image

# Define the visualization parameters for the AOI, such as color (as hex-code)
vis_params = {'color': 'FFD700',
              'pointSize': 3,
              'pointShape': 'circle',
              'width': 2,
              'lineType': 'solid',
              'lineType': 'solid'}

# Add the area of interest as a polygon to the map
Map.addLayer(AOI, vis_params, 'Area of interest')

# Display the map
Map

Map(center=[8.897607, -79.529382], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

---

### Select the data

During the second half of 2023, the Panama Canal area had historically low rainfall. Due to the drought, the water level in the Canal was a lot lower and fewer ships could pass. Therefore, many ships had to wait in front of the entry in Panama City$^{[1]}$. This makes this event a good use case to detect many ships in one image. 
- We select August 2023 as our study period<br>
- We load the Sentinel-1 SAR ImageCollection "S1_GRD"<br>
- We filter to use the VV polarization band. In the polarization mode, the ships act as strong reflectors of the signal (white in the image) while the flat surface of the water acts as a mirror, reflecting the signal away from the sensor (black in the image).<br> 
- We use the Interferometric Wide (IW) swath mode, which is the primary mode from Sentinel-1. Due to the wide swath, it is well suited to monitor large areas.<br>
- Then we filter by the dates we defined and the AOI polygon

In [11]:
# Define study period
startTime = '2023-08-01'
endTime = '2023-08-31'

# Filter the Sentinel-1 ImageCollection by polarisation, instrument mode, date, and area of interest
S1_data = ee.ImageCollection('COPERNICUS/S1_GRD')\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
    .filter(ee.Filter.eq('instrumentMode', 'IW'))\
    .filter(ee.Filter.date(startTime, endTime))\
    .filter(ee.Filter.bounds(AOI))

# Display the number of images available with our filter criteria
print('Number of images in study period:', S1_data.size().getInfo())

Number of images in study period: 4


Inspecting the filtered ImageCollection we see that 4 images are available with our fitler specifications on two different days. One image in the morning and one in the evening. As Sentinel-1 is an active satellite it does not rely on the sunight and can also map the Earth in the dark/with clouds etc.  

In [12]:
S1_data

---

### Generate a ship mask layer

First, we select one image from the ImageCollection (select the first using "first"). Then, we clip it to our AOI polygon (we cut away everything outside the AOI). Last, we only want to store the "VV" band as we need that for the ship detection. To generate the ship mask we use a simple threshold approach. As mentioned above ships have a very high backscatter signal in the SAR image compared to the low background values of the water. Therefore, we can use a threshold value of zero for our binary mask where any pixel with a value greater than zero is considered a ship and assigned a value of 1, and all other pixels are assigned a value of 0.

In [19]:
# Select the first image from the ImageCollection, clip it to the AOI, and only store the "VV" band
S1_image = S1_data.first().clip(AOI).select('VV')

# Make a binary mask (0 and 1) based on a threshold
shipMask = S1_image.gt(0)

Let's visualize both the SAR image and the binary ship mask. Check whether all the detected "ships" are really ships. Where can you see false positives and what could be a reason? How could you improve the result?

In [20]:
# Initialize the map
Map = geemap.Map()

# Define how the basemap should be displayed
Map.set_center(-79.529382, 8.897607, 11) # coordinates and zoom
Map.add_basemap('SATELLITE')             # background map as optical true color image

# Define the visualization parameters for the AOI, such as color (as hex-code)
vis_params = {'color': 'FFD700',
              'pointSize': 3,
              'pointShape': 'circle',
              'width': 2,
              'lineType': 'solid',
              'lineType': 'solid'}

# Define the visualization parameters for the binary ship mask
ship_vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['00000000', 'FF0000'] # from transparent to red
}

# Add the Sentinel-1 image we used for the ship detection
Map.addLayer(S1_image, {'bands': ['VV']}, 'SAR')

# Add the AOI and the ship mask as layers
Map.addLayer(AOI, vis_params, 'Area of interest')
Map.addLayer(shipMask, ship_vis_params, 'Detected Ships')

# Display the map
Map

Map(center=[8.897607, -79.529382], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

---

### Count the number of ships

If we want to count the number of detected ships we first have to connect the positive pixels from the ship mask to bigger "ship-objects", otherwise we would just count the number of pixels with values of 1.

- With the "connectedComponents" function, we can aggregate multiple pixels into one object/ship. The kernel defines how far apart pixels can be to still be considered connected. A kernel of radius 1 is a 3x3 pixel search area. If "ship-pixels" are further apart, they will be considered separate objects.<br>
- To count the number of detected ship-objects, we use the .count() reducer and our defined AOI. This will create a new FeatureCollection with each detected polygon representing one single detected Feature. Using .size() we can get the number of Features in the Collection.

In [39]:
# Connect the "1-pixels" from the ship mask to bigger objects
shipObjects = shipMask.connectedComponents(
    maxSize = 1024,
    connectedness = ee.Kernel.square(radius=1))

# Generate a FeatureCollection with all the ship-objects in the defined AOI
shipCount = shipObjects.reduceToVectors(
    reducer = ee.Reducer.count(),
    geometry = AOI,
    scale = 10,
    maxPixels = 1e13)

# Use .size() to get the number of ships
print('Number of ship-objects detected:', shipCount.size().getInfo())

Number of ship-objects detected: 1813


As you can see the number of ships is probably too hight (1813). But this is also due to the false positives detected in the area of the islands. How could you reduce the number of flasly detected ship objects? Or make it more likely that the detected objects are actually ships?

---

*Literature*<br>
$^{[1]}$: https://www.carbonbrief.org/drought-behind-panama-canals-2023-shipping-disruption-unlikely-without-el-nino/<br>